# Function Duplication

This notebook generates the table for function duplication.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import Version, FuncGroup, FuncGroups, CollisionType, VERSION_DEFAULT
from utils import OUTPUT_PATH
from typing import Dict
import logging


def count_dup(v: Version):
    results: Dict[CollisionType, Dict[str, FuncGroup]] = {c: {} for c in CollisionType}

    func_groups = FuncGroups.from_dump(v.func_groups_path)
    for group in func_groups.iter_groups():
        results[group.collision_type][group.name] = group

    output_path = OUTPUT_PATH / "dup" / v.name
    output_path.mkdir(parents=True, exist_ok=True)
    for c in CollisionType:
        path = output_path / f"{c.name.lower()}.txt"
        with open(path, "w") as f:
            for group in sorted(
                results[c].values(), key=lambda x: x.num_funcs, reverse=True
            ):
                group.print_long(file=f)
        logging.info(f"Saved {len(results[c]):5} groups to {path}")

    return {k: len(v) for k, v in results.items()}


# count_dup(VERSION_DEFAULT)

In [3]:
from depsurf import VersionGroup
from utils import save_pkl

data = {}

for v in VersionGroup.LTS:
    data[v] = count_dup(v)

save_pkl(data, "dup")

[       groups.py:45 ] INFO: Loading funcs from /users/szhong/DepSurf/data/dataset/func_groups/4.4.0-21-generic-amd64.jsonl
[   1919558006.py:23 ] INFO: Saved 17237 groups to /users/szhong/DepSurf/output/dup/4.4.0-21-generic-amd64/unique_global.txt
[   1919558006.py:23 ] INFO: Saved 35710 groups to /users/szhong/DepSurf/output/dup/4.4.0-21-generic-amd64/unique_static.txt
[   1919558006.py:23 ] INFO: Saved  4042 groups to /users/szhong/DepSurf/output/dup/4.4.0-21-generic-amd64/include_dup.txt
[   1919558006.py:23 ] INFO: Saved   404 groups to /users/szhong/DepSurf/output/dup/4.4.0-21-generic-amd64/static_static.txt
[   1919558006.py:23 ] INFO: Saved    10 groups to /users/szhong/DepSurf/output/dup/4.4.0-21-generic-amd64/static_global.txt
[       groups.py:45 ] INFO: Loading funcs from /users/szhong/DepSurf/data/dataset/func_groups/4.15.0-20-generic-amd64.jsonl
[   1919558006.py:23 ] INFO: Saved 20098 groups to /users/szhong/DepSurf/output/dup/4.15.0-20-generic-amd64/unique_global.txt
[ 

In [4]:
from utils import load_pkl, save_latex, center_cell
import pandas as pd


data = load_pkl("dup")

table = {}
for version, dup_counts in data.items():
    col = {}
    col_max = max(dup_counts.values())
    for dup_type, count in dup_counts.items():
        text = f"{count / 1000:.1f}k" if count > 1000 else count
        col[dup_type] = text

    col_name = center_cell(version.short_version)
    table[("Linux Kernel Version", col_name)] = col

df = pd.DataFrame(table)
latex = df.to_latex(multicolumn_format="c", column_format="l|rrrrr")
save_latex(latex, "dup")
df

[ utils_pickle.py:18 ] INFO: Loding dup from /users/szhong/DepSurf/output/dup.pkl
[  utils_latex.py:118] INFO: Saved dup to /users/szhong/DepSurf/paper/tabs/dup.tex


Linux Kernel Version                                                                                                  
                        \multicolumn{1}{c}{4.4} \multicolumn{1}{c}{4.15} \multicolumn{1}{c}{5.4} \multicolumn{1}{c}{5.15} \multicolumn{1}{c}{6.8}
Unique Global                             17.2k                    20.1k                   22.7k                    26.6k                   31.5k
Unique Static                             35.7k                    41.7k                   48.2k                    53.3k                   60.2k
Static Duplication                         4.0k                     4.8k                    5.5k                     6.2k                    7.4k
Static-Static Collision                     404                      398                     411                      444                     498
Static-Global Collision                      10                       26                      27                       26                      29